# Lab 5: Google Speech Commands

## Step 1 - Import and Init

In [ ]:
import copy
import wave
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Input, Conv1D, AvgPool1D, MaxPool1D, ZeroPadding1D, BatchNormalization, Flatten, Dense, Activation
#from keras.utils.data_utils import get_file
#from keras.utils.np_utils import to_categorical
from keras.utils import get_file, to_categorical


In [ ]:
print("Tensor Flow version : " + tf.__version__)
print("Keras version : " + keras.__version__)
print("Numpy version : " + np.__version__)

## Step 2 - Retrieve data
#### Download, cache and extract Google Speech Commands

In [ ]:
# On défini une variable (dataset_dir) pour définir le répertoire où se situe le dataset
dataset_dir = Path('datasets')
# Si le fichier 'testing_list.txt est présent, c'est que le dataset a déjà été téléchargé/extrait 
if not (dataset_dir/'testing_list.txt').exists(): 
    # On télécharge et on extrait le dataset compressé dans le répertoire 'datasets'
    get_file(None, "http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz",
                    extract=True,
                    file_hash="6b74f3901214cb2c2934e98196829835",
                    cache_dir='.',
                    cache_subdir=dataset_dir)

#### Run the following command if needed...

In [ ]:
!mv -f datasets/speech_commands_v0.02.tar.gz/* datasets

In [ ]:
!ls datasets

## <font color="red">**Exo1**</font> : Ecoute de quelques fichiers audio
- Ecoutez quelques fichiers audios avec un lecteur multimédia
- Combien y a t-il de classes au total dans ce dataset ?

<u>Vos réponses</u>:<br>


#### Load raw spoken digits data from Google Speech Commands
Nous n'allons pas chercher à classifier l'ensemble du dataset (ce serait trop long) mais seulement les enregistrements audio des 10 chiffres (0 à 9) 

In [ ]:
# Liste des classes pour le test (ordonnée par label)
CLASSES = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

# La liste 'testing_list' contient le chemin et le nom de tous les échantillons audio (ligne par ligne) pour le test
with (dataset_dir/'testing_list.txt').open() as f:
    # Lecture du fichier (fread)
    # splitlines() : méthode qui divise une chaîne en une liste. La division se fait aux sauts de ligne.
    testing_list = f.read().splitlines()   # ex: right/bb05582b_nohash_3.wav

# On initialise les listes pour l'entrainement et le test (données et labels)
x_train = []
y_train = []
x_test = []
y_test = []

# glob(f'**/*.wav') : permet de lister tous les fichiers audio (extension .wav) présents
# dans l'arborescence du répertoire 'datasets'
for recording in dataset_dir.glob(f'**/*.wav'):
    if not recording.parent.name in CLASSES:       # On ignore les classes qui ne sont pas des chiffres
        continue
    label = CLASSES.index(recording.parent.name)   # On assigne le numéro de la classe à l'enregistrement audio
    
    # Ouverture et lecture des fichiers audio
    with wave.open(str(recording)) as f:           # Read wave file
        # On copie les données audio au format 16-bit signed integer dans un tableau numpy 
        data = np.frombuffer(f.readframes(f.getnframes()), dtype=np.int16).copy()
    
    # Conversion des données audio en format 32-bit floating-point
    data = data.astype(np.float32)
    # Toutes les données audio doivent faire 1 seconde exactement, soit 16000 samples (1 seul canal)
    # On redimensionne si besoin en complétant avec des valeurs à 0 (zero-padding)
    data.resize((16000, 1))

    # if (str(recording.relative_to(dataset_dir))) in testing_list 
    # => NE MARCHE PAS sur mon PC car le chemin est indiqué avec des '\' (Ex: five\20174140_nohash_0.wav) 
    # alors que testing_list utilise des '/'
    # La solution consiste donc à remplacer tous les '\' par des '/'
    
    # On ne met dans le jeu de test que les enregistrements audio listés dans 'testing_list' 
    if (str(recording.relative_to(dataset_dir))).replace("\\", "/") in testing_list: 
        # Assign to test set if file in test list
        x_test.append(data)
        y_test.append(label)
    else:
        x_train.append(data)
        y_train.append(label)

#### Fin de la construction du jeu de données d'entrainement et de test

In [ ]:
x_train = np.array(x_train)
y_train = to_categorical(np.array(y_train))
x_test = np.array(x_test)
y_test = to_categorical(np.array(y_test))

## <font color="red">**Exo2**</font> : Jeu de données
- Observer la forme des données d'entrainement (`x_train`) et de test (`x_test`)
- En déduire le nombre de données pour l'entrainement et pour le test (et donc la taille totale du jeu de données).
- Quel est le pourcentage de données pour le test? et pour l'entrainement ?
- Vérifier le nombre de classes pour l'entrainement et le test?
- Déterminer le nombre d'échantillons/observations d'entrainement pour chaque classe. Est-ce que ce jeu de données est équilibré (en terme de nombre d'échantillons par classe)?

<u>Vos réponses</u>:<br>


## Step 3 - Preparing the data
#### Normalisation des données d'entrainement et de test
+ En retranchant la moyenne et en divisant par l'écart type, on donne à une variable une **moyenne nulle** et un **écart-type de 1**, similaire à une Loi normale centrée réduite .
+ On parle de **normalisation Z-score**
+ Cette transformation assigne à la variable **une majorité de valeurs comprises entre [−1,1]**, le résultat étant **moins dépendant  des valeurs aberrantes** (contrairement à la normalisation min-max).

In [ ]:
# Le jeu d'entrainement est utilisé comme référence pour la moyenne et l'écart type
x_mean = x_train.mean()
x_std = x_train.std()

# Normalisation des données d'entrainement et de test
print('Before normalization : Min={}, Max={}, Moy={}, StdDev={}'.format(x_train.min(), x_train.max(), x_train.mean(), x_train.std()))
x_train -= x_mean
x_test  -= x_mean
x_train /= x_std
x_test  /= x_std
print('After normalization : Min={}, Max={}, Moy={}, StdDev={}'.format(x_train.min(), x_train.max(), x_train.mean(), x_train.std()))

#### Export small dataset (250 random vectors)

In [ ]:
# On mélange aléatoirement les données de test et on prend les 250 premières
perms = np.random.permutation(len(y_test))[0:250]
# On enregistre ces données dans des tableaux numpy et dans un fichier .csv
x_test_250 = x_test[perms]
y_test_250 = y_test[perms]
np.savetxt('x_test_gsc_250.csv', x_test_250.reshape((x_test_250.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_gsc_250.csv', y_test_250, delimiter=',', fmt='%s')

## Step 4 - Build a CNN model with Keras
Ce modèle CNN est issue de cet [article](https://arxiv.org/pdf/1610.00087.pdf).

In [ ]:
model = Sequential()
model.add(Input(shape=(16000, 1)))
model.add(ZeroPadding1D(40))
model.add(Conv1D(filters=128, kernel_size=80, strides=4, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool1D(4))
model.add(ZeroPadding1D(1))
model.add(Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool1D(4))
model.add(ZeroPadding1D(1))
model.add(Conv1D(filters=256, kernel_size=3, strides=1, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool1D(4))
model.add(ZeroPadding1D(1))
model.add(Conv1D(filters=512, kernel_size=3, strides=1, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool1D(4))
model.add(AvgPool1D(15))
model.add(Flatten())
model.add(Dense(units=len(CLASSES)))
model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on

### Let's summarize the constructed model

In [ ]:
model.summary()

## <font color="red">**Exo3**</font>: Etude du modèle CNN
- Combien de couches de convolution/pooling comporte ce modèle CNN ?
- Combien de filtres de convolution sont appliqués aux données audio d'entrée?
- Quelles sont les dimensions des filtres de convolution ?
- Quelle est la taille et le type du filtre de pooling?
- Combien de neurones y-a-t-il à l'entrée du réseau fully-connected (i.e. Dense) ? 
- Combien de couches de neurones sont utilisées dans la partie fully-connected ?
- Combien de neurones par couche sont utilisées dans la partie fully-connected ?
- Combien de paramètres entrainables comportent ce modèle? 
- Retrouver par le calcul ce nombre.

## Build model M5-Smaller
Nous allons utiliser un modèle plus petit, càd avec beaucoup moins de paramètres entrainables que le modèle décrit plus haut...<br>
Nous allons ainsi réduire le temps nécessaire à l'entrainement du modèle tout en gardant de bonnes performances.

In [ ]:
model = Sequential()
model.add(Input(shape=(16000, 1)))
model.add(AvgPool1D(2))
model.add(ZeroPadding1D(20))
model.add(Conv1D(filters=12, kernel_size=40, strides=8, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool1D(4))
model.add(ZeroPadding1D(1))
model.add(Conv1D(filters=12, kernel_size=3, strides=1, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool1D(4))
model.add(ZeroPadding1D(1))
model.add(Conv1D(filters=24, kernel_size=3, strides=1, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool1D(4))
model.add(ZeroPadding1D(1))
model.add(Conv1D(filters=48, kernel_size=3, strides=1, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool1D(4))
model.add(Flatten())
model.add(Dense(units=len(CLASSES)))
model.add(Activation('softmax'))

In [ ]:
model.summary()

## Step 5 - Train the model

#### Let's initialize hyper parameters before training the model

In [ ]:
# optimizer and learning rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-3)
# nb_epochs
nb_epochs = 16  # 50
# taille des lots
batch_size = 192

#### Let's compile the model. 

In [ ]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

On sauvegarde les poids du modèle avant entrainement. On pourra ainsi les recharger plus loin pour recommencer un apprentissage de 0

In [ ]:
model.save_weights('model.weights.h5')

#### Let's train the model

In [ ]:
history = model.fit(x_train, y_train, 
                    epochs=nb_epochs, 
                    batch_size=batch_size, 
                    validation_data=(x_test, y_test))

## Step 6 - Evaluate the model on test dataset

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
print(f'Test loss     : {score[0]:4.4f}')
print(f'Test accuracy : {score[1]*100:4.2f}%')

## <font color="red">**Exo4**</font>: Etude de la performance du CNN
- Quelle précision obtenez-vous sur les données de test avec ce CNN?
- Afficher l'historique de la loss et de l'accuracy en fonction du nombre d'epochs
- Conclure

#### Let's display the history of the loss according to the number of epoch

In [ ]:
# Plot history of the loss
# A COMPLETER

#### Let's display the history of the accuracy according to the number of epoch

In [ ]:
# Plot history of the accuracy
# A COMPLETER

## <font color="red">**Exo5**</font>: Matrice de confusion
+ Afficher la matrice de confusion pour les 10 classes de digits prédites
+ Quels sont les chiffres qui comportent le plus de faux positifs ? Avez-vous une explication ?
+ En vous aidant du TD3, afficher les métriques de précision, recall et f1-score
+ Quelle métrique permet de confirmer le nombre important de faux positifs pour les 2 chiffres indiqués plus haut ? 

In [ ]:
# Affichage de la matrice de confusion
# A COMPLETER...

#### Affichage des métriques : précision, recall et f1-score

In [ ]:
# A COMPLETER

## Evaluate model on small dataset

In [ ]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test_250 = model.predict(x_test_250)
print(tf.math.confusion_matrix(y_test_250.argmax(axis=1), pred_test_250.argmax(axis=1)))

## <font color="red">**Exo6**</font>: 
+ Ré-entrainer le modèle CNN durant 50 epochs cette fois-ci. 
+ Est-ce que les performances (en terme d'accuracy notamment) sont meilleures ? Si oui, que peut-on en conclure ? 

In [ ]:
# Avant de relancer l'entrainement sur 50 epochs, on restaure les poids aléatoires initiaux (avant entrainement)
model.load_weights('model.weights.h5')   

## Sauvegardez le modèle entrainé
N'oubliez pas de sauvegarder votre modèle à la fin de l'entrainement...

In [ ]:
model.save('lab_gsc.h5')

## <font color="red">**Exo7**</font>: Si vous avez le temps...
+ Ré-entrainer le modèle CNN pour classifier les 8 classes suivantes: backward, down, forward, go, left, right, stop, up
+ Evaluer les performances du modèles pour ces 8 classes